# Subreddit CSV creator

This notebook creates a csv of the subreddit with required fields. Only a small filter of number of subscribers more than 2000 is kept

Get the original csv file from here: https://www.kaggle.com/rayraegah/subreddits

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from reddit_crawler import get_default_reddit_inst

In [2]:
csv_file = Path('/scratch/arka/Ark_git_files/subr/subreddits_public.csv')
sub_red_info = pd.read_csv(csv_file)
sub_red_info.subscribers_count = pd.to_numeric(
    sub_red_info.subscribers_count, errors='coerce').fillna(0).astype(np.int_)

In [3]:
min_sub_count = 2000
sub_red_info = sub_red_info[sub_red_info.subscribers_count > min_sub_count]

In [4]:
red_crawl = get_default_reddit_inst()

In [5]:
sub_red_info.head()

,base10_id,base36_reddit_id,creation_epoch,subreddit_name,subscribers_count
0,40718.0,t5_vf2,1.137700e+09,nsfw,936786
3,95487.0,t5_21of,1.140336e+09,olympics,397323
4,96552.0,t5_22i0,1.140417e+09,de,108579
5,96554.0,t5_22i2,1.140417e+09,es,10734
12,98766.0,t5_247i,1.140584e+09,ru,3177


In [6]:
x1 = red_crawl.subreddit(sub_red_info.subreddit_name[0])

In [7]:
x1.allow_videogifs

True

In [15]:
header_list = sub_red_info.columns.tolist() + ['allw_imgs', 'allw_vids', 'allw_gifs']

In [16]:
header_list

['base10_id',
 'base36_reddit_id',
 'creation_epoch',
 'subreddit_name',
 'subscribers_count',
 'allw_imgs',
 'allw_vids',
 'allw_gifs']

In [17]:
sub_red_info = sub_red_info.reindex(columns=header_list)

In [ ]:
i = 0
pain_list = []
for ind, row in tqdm(sub_red_info.iterrows(), total=len(sub_red_info)):
    t = red_crawl.subreddit(row.subreddit_name)
    try:
        row.allw_imgs = t.allow_images
        row.allw_vids = t.allow_videos
        row.allw_gifs = t.allow_videogifs
        sub_red_info.loc[ind] = row
    except:
        i += 1
        pain_list.append(x)
        print(i, x, 'woops')
        row.allw_imgs = True
        row.allw_vids = True
        row.allw_gifs = True
        sub_red_info.loc[ind] = row


In [34]:
sub_red_info

,base10_id,base36_reddit_id,creation_epoch,subreddit_name,subscribers_count,allw_imgs,allw_vids,allw_gifs
0,40718.0,t5_vf2,1.137700e+09,nsfw,936786,True,False,True
3,95487.0,t5_21of,1.140336e+09,olympics,397323,NaN,NaN,NaN
4,96552.0,t5_22i0,1.140417e+09,de,108579,NaN,NaN,NaN
5,96554.0,t5_22i2,1.140417e+09,es,10734,NaN,NaN,NaN
12,98766.0,t5_247i,1.140584e+09,ru,3177,NaN,NaN,NaN
13,113928.0,t5_2fwo,1.141151e+09,programming,1059670,NaN,NaN,NaN
15,199566.0,t5_49zi,1.144811e+09,freeculture,7494,NaN,NaN,NaN
19,1058648.0,t5_mouw,1.161180e+09,science,18641755,NaN,NaN,NaN
20,2156921.0,t5_1a8ah,1.173892e+09,obama,23078,NaN,NaN,NaN
21,2352663.0,t5_1efbr,1.175413e+09,ads,9689,NaN,NaN,NaN
